# Loading Pan01 DSW export files

In [1]:
import cosima_cookbook as cc
import glob

import numpy as np
import netCDF4 as nc
import xarray as xr
from gsw import SA_from_SP, p_from_z, sigma1
import sys,os
from pathlib import Path

In [2]:
import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

In [3]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 125.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37333,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 125.19 GiB
Comm: tcp://127.0.0.1:44109,Total threads: 4
Dashboard: /proxy/42825/status,Memory: 17.88 GiB
Nanny: tcp://127.0.0.1:33719,


In [4]:
# comment/uncomment the path to the expt to choose which one
expt_folder = '/scratch/x77/jn8053/mom6/archive/panant-01-zstar-ssp126-prsc-rest/'
expt = 'panant-01-zstar-ssp126-prsc-rest'
# choose a location to save - ideally on gdata
save_dir = '/g/data/x77/kc5856/sofia-cross_slope_transport/'+expt+'/Antarctic_cross_slope/'
path_database = '/g/data/g40/kc5856/database/'

In [5]:
expt_name = 'panan_01deg_preM'
db = path_database + 'honours_prescribed.db' # change if/when needed!
session = cc.database.create_session(db)
frequency = '1 monthly'

In [6]:
# reference density in MOM6 
rho_0 = 1035.0
lat_range = slice(-90,-59) 

In [7]:
resolution = expt_name.split('_')[1][:-3]
contour_depth = 1000
# Open contour data
ds_contour = xr.open_dataset('/g/data/ik11/grids/Antarctic_slope_contour_' + str(contour_depth) + 'm_MOM6_' + resolution + 'deg.nc')

In [8]:
# load data and rename coordinates to general x/y to be able to multiply them
mask_y_transport = ds_contour.mask_y_transport.rename({'yq': 'y', 'xh': 'x'})
mask_x_transport = ds_contour.mask_x_transport.rename({'yh': 'y', 'xq': 'x'})
mask_y_transport_numbered = ds_contour.mask_y_transport_numbered.rename({'yq': 'y', 'xh': 'x'})
mask_x_transport_numbered = ds_contour.mask_x_transport_numbered.rename({'yh': 'y', 'xq': 'x'})

# number of points along contour:
num_points = int(np.maximum(np.max(mask_y_transport_numbered),np.max(mask_x_transport_numbered)))

In [9]:
##### Stack contour data into 1D and extract lat/lon on contour
# Create the contour order data-array. Note that in this procedure the
# x-grid counts have x-grid dimensions and the y-grid counts have y-grid
# dimensions, but these are implicit, the dimension *names* are kept
# general across the counts, the generic y/x, so that concatening
# works but we dont double up with numerous counts for one lat/lon point.

# stack contour data into 1d:
mask_x_numbered_1d = mask_x_transport_numbered.stack(contour_index = ['y', 'x'])
mask_x_numbered_1d = mask_x_numbered_1d.where(mask_x_numbered_1d > 0, drop = True)
mask_y_numbered_1d = mask_y_transport_numbered.stack(contour_index = ['y', 'x'])
mask_y_numbered_1d = mask_y_numbered_1d.where(mask_y_numbered_1d > 0, drop = True)
contour_ordering = xr.concat((mask_x_numbered_1d,mask_y_numbered_1d), dim = 'contour_index')
contour_ordering = contour_ordering.sortby(contour_ordering)

# get lat and lon along contour, useful for plotting later:
lat_along_contour = contour_ordering.y
lon_along_contour = contour_ordering.x
contour_index_array = np.arange(1,len(contour_ordering)+1)

# don't need the multi-index anymore, replace with contour count and save
lat_along_contour = lat_along_contour.drop_vars({'x', 'y', 'contour_index'})
lat_along_contour.coords['contour_index'] = contour_index_array
lon_along_contour = lon_along_contour.drop_vars({'x', 'y', 'contour_index'})
lon_along_contour.coords['contour_index'] = contour_index_array

In [34]:
year = '2010'
start_time= year + '-01-01'
end_time= year + '-12-31'

In [35]:
#### Load mass transport umo and vmo
vmo = cc.querying.getvar(
    expt, 'vmo', session, frequency=frequency,
    start_time=start_time, end_time=end_time,
    chunks={'xh': '200MB', 'yq': '200MB'})

umo = cc.querying.getvar(
    expt, 'umo', session, frequency=frequency,
    start_time=start_time, end_time=end_time,
    chunks={'xq': '200MB', 'yh': '200MB'})

# select latitude range and this year:
vmo = vmo.sel(yq=lat_range).sel(time=slice(start_time,end_time))
vmo = vmo.isel(yq=slice(1, None))
umo = umo.sel(yh=lat_range).sel(time=slice(start_time,end_time))
umo = umo.isel(xq=slice(1, None))

# Note that vmo is Ocean Mass Y Transport (kg s-1) and defined as the transport across
# the northern edge of a tracer cell so its coordinates should be (yq, xh).
# umo is Ocean Mass X Transport (kg s-1) and defined as the transport across
# the eastern edge of a tracer cell so its coordinates should be (yh, xq).
# However we will keep the actual name as simply y/x irrespective of the variable
# to make concatenation and sorting possible.
vmo = vmo.rename({'yq':'y', 'xh':'x'})
umo = umo.rename({'yh':'y', 'xq':'x'})

# convert kg/s to Sv and multiply by contour masks
vmo = vmo/(1e6*rho_0)*mask_y_transport
umo = umo/(1e6*rho_0)*mask_x_transport

In [36]:
#### Extract transport values along contour
umo_i = umo.compute()

In [37]:
#### Extract transport values along contour
vmo_i = vmo.compute()

In [38]:
# stack transports into 1d and drop any points not on contour:
x_transport_1d_i = umo_i.stack(contour_index=['y', 'x'])
x_transport_1d_i = x_transport_1d_i.where(mask_x_numbered_1d>0, drop=True)
y_transport_1d_i = vmo_i.stack(contour_index=['y', 'x'])
y_transport_1d_i = y_transport_1d_i.where(mask_y_numbered_1d>0, drop=True)
        
# combine all points on contour:
vol_trans_across_contour = xr.concat((x_transport_1d_i, y_transport_1d_i), dim='contour_index')
vol_trans_across_contour = vol_trans_across_contour.sortby(contour_ordering)
vol_trans_across_contour = vol_trans_across_contour.drop_vars({'x', 'contour_index', 'y'})
vol_trans_across_contour.coords['contour_index'] = contour_index_array
vol_trans_across_contour = vol_trans_across_contour.compute()

In [39]:
#### save data
vol_trans_across_contour.name = 'vol_trans_across_contour'
vol_trans_across_contour.attrs = {
    'long_name': 'Volume transport across' + str(contour_depth) +
    'm isobath',
    'units': 'Sv'}
ds = vol_trans_across_contour.to_dataset()
ds['lat'] = lat_along_contour
ds['lon'] = lon_along_contour
if len(vol_trans_across_contour.contour_index) < 15000:
    chunk_ind = len(vol_trans_across_contour.contour_index)
else:
    chunk_ind = 10000
enc = {'vol_trans_across_contour':
       {'chunksizes': (len(vol_trans_across_contour.time), 99, chunk_ind),
        'zlib': True, 'complevel': 5, 'shuffle': True}}

ds.to_netcdf(save_dir + 'vol_transp_across_' + str(contour_depth) +
             'm_isobath_' + expt_name + '_' + frequency[:3:2] + '_' +
             year + '.nc', encoding=enc)